In [ ]:
# Import required packages
import sys
# !{sys.executable} -m pip install seaborn
from glob import glob
import os
import json
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from shutil import copy,move
from distutils.dir_util import copy_tree


In [ ]:
def results_df(path):
    '''
    Take the top directory where the results are and return a nice Pandas dataframe

    parameters
    ----------
    path: str
        Path to the top directory `path/*/*results.json`

    returns
    -------
    df: pd.DataFrame
        Dataframe containing results
    '''
    results_dict = {}
    for i in glob(os.path.join(path,'*/*results.json')):
        with open(i,'r') as f:
            results = json.load(f)
        name = os.path.basename(i).replace('_results.json','')
        results['encoder']=name.split('_')[0]
        results['ansatz']=name.split('_')[1]
        results_dict[name]=results

    df = pd.DataFrame(results_dict).T
    
    return df



In [ ]:
def plot_all(df,name,save_dir):
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(12,6), gridspec_kw={'width_ratios': [1, 1]})
    SMALL_SIZE = 8
    MEDIUM_SIZE = 12
    BIGGER_SIZE = 12
    figsize = fig.get_size_inches()
    fig.set_size_inches(figsize * 1.2)
    plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title
    
    
    sns.boxplot(data=df,x='encoder',y='value',hue='variable',ax=ax1,palette=sns.color_palette('Paired',2))
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=90) 
    ax1.set_ylim(np.floor(df['value'].min()),np.ceil(df['value'].max()))
    ax1.set_ylabel('R$^{2}$')
    ax1.set_xlabel('Encoder')
    ax1.legend(title='')
    
    sns.boxplot(data=df,x='ansatz',y='value',hue='variable',ax=ax2,palette=sns.color_palette('Paired',2))
    ax2.set_xticklabels(ax2.get_xticklabels(), rotation=90) 
    ax2.set_ylim(np.floor(df['value'].min()),np.ceil(df['value'].max()))
    ax2.set_ylabel('R$^{2}$')
    ax2.set_xlabel('Ansatz')
    ax2.legend(title='')
    plt.tight_layout()
    plt.savefig(os.path.join(os.path.expanduser('~'),save_dir,f'{name}_boxplots.png'),dpi=300,bbox_inches='tight')
    plt.show()
    
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(13,6),sharey=False)
    SMALL_SIZE = 8
    MEDIUM_SIZE = 12
    BIGGER_SIZE = 12
    
    plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title
    figsize = fig.get_size_inches()
    fig.set_size_inches(figsize * 1.5)
    ax1=sns.heatmap(df[df['variable']=='Train'][["encoder", "ansatz", "value"]].pivot(index='encoder', columns='ansatz', values='value').astype(float),annot_kws={"size": 12}, square=True, annot=True, linewidth=.5,fmt='.2f',ax=ax1,vmax=1,vmin=-1,cbar=True,center=0,cmap='rocket')
    ax1.set_title("Train")
    ax1.set_ylabel('Encoder')
    ax1.set_xlabel('Ansatz')
    ax1.set_aspect("equal")
    
    ax2=sns.heatmap(df[df['variable']=='Test'][["encoder", "ansatz", "value"]].pivot(index='encoder', columns='ansatz', values='value').astype(float), annot_kws={"size": 12}, square=True, annot=True, linewidth=.5,fmt='.2f',ax=ax2,vmax=1,vmin=-1,center=0,cmap='rocket')
    ax2.set_title("Test")
    ax2.set_ylabel('Encoder')
    ax2.set_xlabel('Ansatz')
    ax2.set_aspect("equal")
    
    ax1.collections[0].colorbar.set_label('R$^{2}$')
    ax2.collections[0].colorbar.set_label('R$^{2}$')

    plt.subplots_adjust(wspace=0.1,hspace=0)
    plt.tight_layout()  # Leave space for color bar
    plt.savefig(os.path.join(os.path.expanduser('~'),save_dir,f'{name}_heatplots.png'),dpi=300,bbox_inches='tight')
    plt.show()

In [ ]:
def plot(data_dir,name,save_dir):
    df = results_df(data_dir).rename(columns={'R2_train':'Train','R2_test':'Test'}).melt(id_vars=['encoder','ansatz'], value_vars=['Train','Test']).sort_values(by=['encoder','ansatz'])
    plot_all(df,name,save_dir)    

In [ ]:
plot("function-calc-test/fivequbit_tests/lin5qubit/",'linear','qregress/images/Function_Fitting/fivequbit/')
plot("function-calc-test/fivequbit_tests/quad5qubit/",'quadratic','qregress/images/Function_Fitting/fivequbit/')
plot("function-calc-test/fivequbit_tests/sin5qubit/",'sine','qregress/images/Function_Fitting/fivequbit/')

In [ ]:
plot("function-calc-test/sixteenqubit_test/lin16qubit/",'linear','qregress/images/Function_Fitting/sixteenqubit/')
plot("function-calc-test/sixteenqubit_test/quad16qubit/",'quadratic','qregress/images/Function_Fitting/sixteenqubit/')
plot("function-calc-test/sixteenqubit_test/sin16qubit/",'sine','qregress/images/Function_Fitting/sixteenqubit/')

In [ ]:
plot("BSE-calc-test/5qubit/",'BSE','qregress/images/BSE/fivequbit/')
plot("BSE-calc-test/16qubit/",'BSE','qregress/images/BSE/sixteenqubit/')
plot("qml_DDCC/5/",'DDCC','qregress/images/DDCC/')

In [ ]:

checkbse5 = results_df("BSE-calc-test/5qubit/").rename(columns={'R2_train':'Train','R2_test':'Test'}).melt(id_vars=['encoder','ansatz'], value_vars=['Train','Test']).sort_values(by=['encoder','ansatz'])

In [ ]:
results_df("BSE-calc-test/5qubit/").sort_values(by=['R2_train','R2_test'])

In [ ]:
M-M-CZ_HWE-CNOT

In [ ]:
checkbse5[checkbse5['encoder']=='A1-A1-CZ']['value'].astype(float).describe()

In [ ]:
# [(i,checkbse5[checkbse5['encoder']==i]['value'].astype(float)) for i in set(checkbse5['encoder'])]
pd.concat([checkbse5[checkbse5['encoder']==i]['value'].astype(float).to_frame().describe().rename(columns={'value':i}) for i in set(checkbse5['encoder'])],axis=1).T.sort_values(by=['mean'])

In [ ]:
pd.concat([checkbse5[checkbse5['ansatz']==i]['value'].astype(float).to_frame().describe().rename(columns={'value':i}) for i in set(checkbse5['ansatz'])],axis=1).T.sort_values(by=['mean'])

In [ ]:
sorted([(i,f"{checkbse5[checkbse5['encoder']==i]['value'].astype(float).mean():.4f}") for i in set(checkbse5['encoder'])],key=lambda x: x[1])

In [ ]:
sorted([(i,f"{checkbse5[checkbse5['ansatz']==i]['value'].astype(float).mean():.4f}") for i in set(checkbse5['ansatz'])],key=lambda x: x[1])

In [ ]:
def plot_combined(df,name,save_dir):


    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(12,6), gridspec_kw={'width_ratios': [1, 1]})
    SMALL_SIZE = 8
    MEDIUM_SIZE = 12
    BIGGER_SIZE = 12
    figsize = fig.get_size_inches()
    fig.set_size_inches(figsize * 1.2)
    plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
    plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
    plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=MEDIUM_SIZE)    # fontsize of the tick labels
    plt.rc('legend', fontsize=MEDIUM_SIZE)    # legend fontsize
    plt.rc('figure', titlesize=MEDIUM_SIZE)  # fontsize of the figure title
    
    
    sns.boxplot(data=df,x='encoder',y='value',hue='variable',ax=ax1,palette=sns.color_palette('Paired',2))
    ax1.set_xticklabels(ax1.get_xticklabels(), rotation=90) 
    ax1.set_ylim(np.floor(df['value'].min()),np.ceil(df['value'].max()))
    ax1.set_ylabel('R$^{2}$')
    ax1.set_xlabel('Encoder')
    ax1.legend(title='')
    
    sns.boxplot(data=df,x='ansatz',y='value',hue='variable',ax=ax2,palette=sns.color_palette('Paired',2))
    ax2.set_xticklabels(ax2.get_xticklabels(), rotation=90) 
    ax2.set_ylim(np.floor(df['value'].min()),np.ceil(df['value'].max()))
    ax2.set_ylabel('R$^{2}$')
    ax2.set_xlabel('Ansatz')
    ax2.legend(title='')
    plt.tight_layout()
    plt.savefig(os.path.join(os.path.expanduser('~'),save_dir,f'{name}_boxplots.png'),dpi=300,bbox_inches='tight')
    plt.show()    

In [ ]:
concatfive = pd.concat([results_df("function-calc-test/fivequbit_tests/lin5qubit/"),results_df("function-calc-test/fivequbit_tests/quad5qubit/"),results_df("function-calc-test/fivequbit_tests/sin5qubit/")]).rename(columns={'R2_train':'Train','R2_test':'Test'}).melt(id_vars=['encoder','ansatz'], value_vars=['Train','Test']).sort_values(by=['encoder','ansatz'])

In [ ]:
lin5 = results_df("function-calc-test/fivequbit_tests/lin5qubit/").rename(columns={'R2_train':'Train','R2_test':'Test'}).melt(id_vars=['encoder','ansatz'], value_vars=['Train','Test']).sort_values(by=['encoder','ansatz'])
quad5 = results_df("function-calc-test/fivequbit_tests/quad5qubit/").rename(columns={'R2_train':'Train','R2_test':'Test'}).melt(id_vars=['encoder','ansatz'], value_vars=['Train','Test']).sort_values(by=['encoder','ansatz'])
sin5 = results_df("function-calc-test/fivequbit_tests/sin5qubit/").rename(columns={'R2_train':'Train','R2_test':'Test'}).melt(id_vars=['encoder','ansatz'], value_vars=['Train','Test']).sort_values(by=['encoder','ansatz'])

In [ ]:
check5 = results_df("function-calc-test/fivequbit_tests/lin5qubit//")
check5[(check5['R2_train']>=0.988)&(check5['R2_test']>=0.988)].sort_values(by=['R2_test','R2_train'])

In [ ]:
sorted([(i,f"{concatfive[concatfive['encoder']==i]['value'].astype(float).mean():.4f}") for i in set(concatfive['encoder'])],key=lambda x: x[1])

In [ ]:
sorted([(i,f"{concatfive[concatfive['ansatz']==i]['value'].astype(float).mean():.4f}") for i in set(concatfive['ansatz'])],key=lambda x: x[1])

In [ ]:
plot_combined(concatfive,'five_feature_function_fitting','qregress/images/Function_Fitting/fivequbit/')

In [ ]:
concatsixteen = pd.concat([results_df("function-calc-test/sixteenqubit_test/lin16qubit/"),results_df("function-calc-test/sixteenqubit_test/quad16qubit//"),results_df("function-calc-test/sixteenqubit_test/sin16qubit/")]).rename(columns={'R2_train':'Train','R2_test':'Test'}).melt(id_vars=['encoder','ansatz'], value_vars=['Train','Test']).sort_values(by=['encoder','ansatz'])

In [ ]:
lin16 = results_df("function-calc-test/sixteenqubit_test/lin16qubit/").rename(columns={'R2_train':'Train','R2_test':'Test'}).melt(id_vars=['encoder','ansatz'], value_vars=['Train','Test']).sort_values(by=['encoder','ansatz'])
quad16 = results_df("function-calc-test/sixteenqubit_test/quad16qubit//").rename(columns={'R2_train':'Train','R2_test':'Test'}).melt(id_vars=['encoder','ansatz'], value_vars=['Train','Test']).sort_values(by=['encoder','ansatz'])
sin16 = results_df("function-calc-test/sixteenqubit_test/sin16qubit/").rename(columns={'R2_train':'Train','R2_test':'Test'}).melt(id_vars=['encoder','ansatz'], value_vars=['Train','Test']).sort_values(by=['encoder','ansatz'])

In [ ]:
check = results_df("function-calc-test/sixteenqubit_test/lin16qubit//")
check[(check['R2_train']>=0.988)&(check['R2_test']>=0.988)].sort_values(by=['R2_test','R2_train'])

In [ ]:
sorted([(i,f"{sin16[sin16['encoder']==i]['value'].astype(float).mean():.4f}") for i in set(sin16['encoder'])],key=lambda x: x[1])

In [ ]:
sorted([(i,f"{sin16[sin16['ansatz']==i]['value'].astype(float).mean():.4f}") for i in set(sin16['ansatz'])],key=lambda x: x[1])

In [ ]:
plot_combined(concatsixteen,'sixteen_feature_function_fitting','qregress/images/Function_Fitting/sixteenqubit/')

In [ ]:
'images/Function_Fitting/sixteenqubit/sixteen_feature_function_fitting_boxplots.png'